In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import folium
from folium import plugins
import datetime as dt
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter, YearLocator

%matplotlib inline 
plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', None)

# Import data

In [2]:
df = pd.read_csv('data/incidenti_2007_2017.csv')

/Users/edoardoventurini/Envs/python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df[df['DataOraIncidente_year']!=2017].copy()

In [4]:
df.head()

,Chilometrica,CondizioneAtmosferica,Confermato,DaSpecificare,DataOraIncidente,FondoStradale,Gruppo,ID,Illuminazione,Latitudine,Localizzazione1,Localizzazione2,Longitudine,NUM_FERITI,NUM_ILLESI,NUM_MORTI,NUM_RISERVATA,NaturaIncidente,ParticolaritaStrade,Pavimentazione,Segnaletica,Strada02,Strada1,Strada2,TipoStrada,Traffico,Visibilita,DataOraIncidente_date,DataOraIncidente_time,DataOraIncidente_time_h,DataOraIncidente_weekday,DataOraIncidente_month,DataOraIncidente_year
0,151-153,Nuvoloso,1.0,NaN,2007-01-01 00:00:00,Asciutto,18,1707637,Sufficiente,NaN,Strada Urbana,in prossimità,NaN,0,0,0,0,Veicolo in marcia contro veicolo in sosta,Rettilineo,Asfaltata,Verticale ed orizzontale,del civico n.,VIA ANASTASIO II,NaN,Due carreggiate,Normale,Buona,2007-01-01,00:00:00,0,0,1,2007
1,537,Nuvoloso,1.0,NaN,2007-01-01 00:40:00,Asciutto,4,1708657,Sufficiente,NaN,Provinciale entro l'abitato,in prossimità,NaN,2,9,0,0,Tamponamento Multiplo,Rettilineo,Asfaltata,Verticale ed orizzontale,del civico n.,VIA NOMENTANA,NaN,Più di due carreggiate,Normale,Buona,2007-01-01,00:40:00,0,0,1,2007
2,123/A,Pioggia in atto,1.0,NaN,2007-01-01 01:10:00,Bagnato (pioggia),18,1707214,Sufficiente,NaN,Strada Urbana,in prossimità,NaN,1,2,0,0,Scontro frontale/laterale DX fra veicoli in ma...,Rettilineo,Asfaltata,Orizzontale,del civico n.,VIA MATTIA BATTISTINI,NaN,Una carreggiata a doppio senso,Normale,Buona,2007-01-01,01:10:00,1,0,1,2007
3,13,Nebbia,0.0,NaN,2007-01-01 01:30:00,Bagnato (umidità in atto),5,1707689,Sufficiente,NaN,Strada Urbana,in prossimità,NaN,0,1,0,0,Veicolo in marcia contro veicoli fermi,Rettilineo,Asfaltata,Verticale,del civico n.,VIA CESARE DE LOLLIS,NaN,Una carreggiata a senso unico di marcia,Normale,Sufficiente,2007-01-01,01:30:00,1,0,1,2007
4,NaN,Nuvoloso,1.0,NaN,2007-01-01 01:30:00,Asciutto,16,1708159,Sufficiente,NaN,Strada Urbana,all'intersezione con,NaN,0,2,0,0,Scontro frontale/laterale SX fra veicoli in ma...,Incrocio,In cubetti di porfido,Verticale ed orizzontale,VIA RAFFAELLO GIOVAGNOLI,VIALE DEI QUATTRO VENTI,VIA RAFFAELLO GIOVAGNOLI,Una carreggiata a doppio senso,Normale,Buona,2007-01-01,01:30:00,1,0,1,2007


In [5]:
len(df)

344040

In [6]:
df['DataOraIncidente'] = df['DataOraIncidente'].apply(lambda d: pd.to_datetime(d))

# Create the dataset

In [38]:
df['is_weekend'] = df['DataOraIncidente_weekday'].apply(lambda x: 1 if x>=5 else 0)

In [7]:
df.columns

Index(['Chilometrica', 'CondizioneAtmosferica', 'Confermato', 'DaSpecificare',
       'DataOraIncidente', 'FondoStradale', 'Gruppo', 'ID', 'Illuminazione',
       'Latitudine', 'Localizzazione1', 'Localizzazione2', 'Longitudine',
       'NUM_FERITI', 'NUM_ILLESI', 'NUM_MORTI', 'NUM_RISERVATA',
       'NaturaIncidente', 'ParticolaritaStrade', 'Pavimentazione',
       'Segnaletica', 'Strada02', 'Strada1', 'Strada2', 'TipoStrada',
       'Traffico', 'Visibilita', 'DataOraIncidente_date',
       'DataOraIncidente_time', 'DataOraIncidente_time_h',
       'DataOraIncidente_weekday', 'DataOraIncidente_month',
       'DataOraIncidente_year'],
      dtype='object')

In [72]:
features = [
            'is_weekend',
            #'Chilometrica', 
            'CondizioneAtmosferica', 
            #'Confermato', 
            #'DaSpecificare',
            #'DataOraIncidente', 
            'FondoStradale', 
            #'Gruppo', 
            #'ID', 
            'Illuminazione',
            #'Latitudine', 
            #'Localizzazione1', 
            #'Localizzazione2', 
            #'Longitudine',
            #'NUM_FERITI', 
            #'NUM_ILLESI', 
            #'NUM_MORTI',    @@@@@
            #'NUM_RISERVATA',
            #'NaturaIncidente', @@@@@@@@@@@@@@
            'ParticolaritaStrade',
            'Pavimentazione',
            'Segnaletica', 
            #'Strada02',
            #'Strada1', 
            #'Strada2',
            'TipoStrada',
            'Traffico', 
            'Visibilita', 
            #'DataOraIncidente_date',
            #'DataOraIncidente_time', 
            #'DataOraIncidente_time_h',
            #'DataOraIncidente_weekday',
            #'DataOraIncidente_month',
            #'DataOraIncidente_year'
            ]

In [73]:
features2values = {}
for f in features:
    print(f)
    values = df.groupby(f).count()[['ID']].sort_values(by='ID',ascending=False)/len(df)
    top_values = values[values['ID']>=0.02].index.values
    for v in top_values:
        print('    '+str(v))
    print()
    features2values[f] = top_values.tolist()

is_weekend
    0
    1

CondizioneAtmosferica
    Sereno
    Nuvoloso
    Pioggia in atto

FondoStradale
    Asciutto
    Bagnato (pioggia)
    Bagnato (umidità in atto)

Illuminazione
    Ore Diurne
    Sufficiente
    other

ParticolaritaStrade
    Rettilineo
    Incrocio
    Intersezione semaforizzata
    Curva a visuale libera
    other
    Intersezione stradale segnalata
    Curva senza visuale libera

Pavimentazione
    Asfaltata
    other

Segnaletica
    Verticale ed orizzontale
    Orizzontale
    Assente
    Verticale

TipoStrada
    Una carreggiata a doppio senso
    Una carreggiata a senso unico di marcia
    Due carreggiate
    Più di due carreggiate

Traffico
    Normale
    Scarso
    Intenso
    other

Visibilita
    Buona
    Sufficiente
    other



In [74]:
def create_other(mydf, column, values_to_save=[]):
    if len(values_to_save)==0:
        return mydf
    mydf[column] = mydf[column].apply(lambda x: x if x in values_to_save else 'other')
    return mydf

In [75]:
features2values

{'CondizioneAtmosferica': ['Sereno', 'Nuvoloso', 'Pioggia in atto'],
 'FondoStradale': ['Asciutto',
  'Bagnato (pioggia)',
  'Bagnato (umidità in atto)'],
 'Illuminazione': ['Ore Diurne', 'Sufficiente', 'other'],
 'ParticolaritaStrade': ['Rettilineo',
  'Incrocio',
  'Intersezione semaforizzata',
  'Curva a visuale libera',
  'other',
  'Intersezione stradale segnalata',
  'Curva senza visuale libera'],
 'Pavimentazione': ['Asfaltata', 'other'],
 'Segnaletica': ['Verticale ed orizzontale',
  'Orizzontale',
  'Assente',
  'Verticale'],
 'TipoStrada': ['Una carreggiata a doppio senso',
  'Una carreggiata a senso unico di marcia',
  'Due carreggiate',
  'Più di due carreggiate'],
 'Traffico': ['Normale', 'Scarso', 'Intenso', 'other'],
 'Visibilita': ['Buona', 'Sufficiente', 'other'],
 'is_weekend': [0, 1]}

In [76]:
for f,values in features2values.items():
    df = create_other(mydf=df, column=f, values_to_save=values)

In [77]:
df_new = df[features].copy().reset_index(drop=True)
df_new['target'] = (df['NUM_MORTI']>0).astype(int)
for f in features2values.keys():
    df_new = pd.merge(df_new,
                      pd.get_dummies(df_new[f],prefix=f),
                      left_index=True,
                      right_index=True)  
    df_new.drop(f, axis=1, inplace=True)

In [78]:
df_new.head()

,target,is_weekend_0,is_weekend_1,Visibilita_Buona,Visibilita_Sufficiente,Visibilita_other,CondizioneAtmosferica_Nuvoloso,CondizioneAtmosferica_Pioggia in atto,CondizioneAtmosferica_Sereno,CondizioneAtmosferica_other,Pavimentazione_Asfaltata,Pavimentazione_other,Traffico_Intenso,Traffico_Normale,Traffico_Scarso,Traffico_other,ParticolaritaStrade_Curva a visuale libera,ParticolaritaStrade_Curva senza visuale libera,ParticolaritaStrade_Incrocio,ParticolaritaStrade_Intersezione semaforizzata,ParticolaritaStrade_Intersezione stradale segnalata,ParticolaritaStrade_Rettilineo,ParticolaritaStrade_other,TipoStrada_Due carreggiate,TipoStrada_Più di due carreggiate,TipoStrada_Una carreggiata a doppio senso,TipoStrada_Una carreggiata a senso unico di marcia,TipoStrada_other,FondoStradale_Asciutto,FondoStradale_Bagnato (pioggia),FondoStradale_Bagnato (umidità in atto),FondoStradale_other,Illuminazione_Ore Diurne,Illuminazione_Sufficiente,Illuminazione_other,Segnaletica_Assente,Segnaletica_Orizzontale,Segnaletica_Verticale,Segnaletica_Verticale ed orizzontale,Segnaletica_other
0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
1,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
2,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0
3,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0
4,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0


In [79]:
df_new.groupby('target').count()[['is_weekend_0']]

,is_weekend_0
target,
0,342494
1,1546


In [80]:
df_new.groupby('target').count()[['is_weekend_0']]/len(df_new)

,is_weekend_0
target,
0,0.995506
1,0.004494


In [60]:
from sklearn.linear_model import LogisticRegression

In [81]:
features = [c for c in df_new.columns if c!='target']

lr = LogisticRegression()
lr.fit(df_new[features], df_new['target'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [85]:
feat2coef = pd.DataFrame()
feat2coef['feature'] = features
feat2coef['coeff'] = lr.coef_[0]

feat2coef.sort_values(by='coeff')

,feature,coeff
10,Pavimentazione_other,-1.137219
19,ParticolaritaStrade_Intersezione stradale segn...,-1.100755
30,FondoStradale_other,-1.077195
31,Illuminazione_Ore Diurne,-1.022048
34,Segnaletica_Assente,-0.926156
0,is_weekend_0,-0.805742
17,ParticolaritaStrade_Incrocio,-0.804067
6,CondizioneAtmosferica_Pioggia in atto,-0.783723
2,Visibilita_Buona,-0.743721
26,TipoStrada_other,-0.733594
